In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

import kerastuner as kt

# Get data

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# normalize
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Create model

In [3]:
def model_builder(hp): # hp stands for hyperparameter
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28))) # convert to 1D

    # tune the number of units for first dense layer, between 32 - 512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    
    # output layer
    model.add(keras.layers.Dense(10))

    # tune learning rate for optimizer, within 0.01, 0.001, 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])

    return model

# Hypertuning

In [4]:
# instantiate keras tuner
tuner = kt.Hyperband(
    model_builder, # the model to be tuned, the hypermodel
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='kt'
)

In [5]:
# a callback to clear training outputs after every training step
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [6]:
# run hyperparameter tuning
tuner.search(img_train, label_train, epochs=10, validation_data=(img_test, label_test), callbacks=[ClearTrainingOutput()])

# the optimal hp
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.001.



In [7]:
# use the best hp to build model and train
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4879 - accuracy: 0.8264 - val_loss: 0.4534 - val_accuracy: 0.8416
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3690 - accuracy: 0.8673 - val_loss: 0.4021 - val_accuracy: 0.8543
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3324 - accuracy: 0.8783 - val_loss: 0.3553 - val_accuracy: 0.8737
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3089 - accuracy: 0.8853 - val_loss: 0.3594 - val_accuracy: 0.8702
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2909 - accuracy: 0.8929 - val_loss: 0.3561 - val_accuracy: 0.8703
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2761 - accuracy: 0.8974 - val_loss: 0.3714 - val_accuracy: 0.8641
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2642 - accuracy: 0.9024 - val_loss: 0.3368 - val_accuracy: